In [7]:
import numpy as np
import qutip as qt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, Aer
from qiskit_aer import StatevectorSimulator
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import Statevector, random_statevector, partial_trace
from qiskit.circuit import Parameter
from qiskit.circuit.library import QFT
import random
from time import time
from typing import Optional

from tools.quantum import *
from tools.classical import *
from boltzmann import *

In [8]:
# np.random.seed(666)
num_energy_bits = 4
beta = 1
bitstrings = [bin(i)[2:].zfill(num_energy_bits - 1) for i in range(2**(num_energy_bits - 1))]
bitstring = bitstrings[np.random.randint(0, len(bitstrings))]
omega = int(bitstring, 2) / 2**(num_energy_bits - 1)
boltzmann_weight = lambda w: np.exp(- w * beta)
# W if |1> is 'accept'
W_matrix = np.array([[np.sqrt(1 - boltzmann_weight(omega)), np.sqrt(boltzmann_weight(omega))],
                     [np.sqrt(boltzmann_weight(omega)), -np.sqrt(1 - boltzmann_weight(omega))]])

print("omega:", omega)
print("boltzmann weight:", boltzmann_weight(omega))
print("W matrix:", W_matrix)
accept_state = np.array([0, 1])
reject_state = np.array([1, 0])

initial_state = np.array([1, 0])
end_state = W_matrix @ initial_state
print("end state:", end_state)
print(f'Accept probability: {end_state[1] ** 2}')
print(f'Reject probability: {end_state[0] ** 2}')



omega: 0.625
boltzmann weight: 0.5352614285189903
W matrix: [[ 0.68171737  0.73161563]
 [ 0.73161563 -0.68171737]]
end state: [0.68171737 0.73161563]
Accept probability: 0.5352614285189903
Reject probability: 0.4647385714810097


In [9]:
from qiskit.circuit.library import RXGate, RYGate, CXGate
W_op = Operator(W_matrix)
circ = QuantumCircuit(1)
# circ.x(0)  # To undo the cx from the default case that energy is negative
circ.append(W_op, [0])
tr_circ = transpile(circ, basis_gates=['rx', 'ry', 'cx'])
print(tr_circ)


global phase: 3π/2
   ┌────────┐┌─────────────┐
q: ┤ Rx(-π) ├┤ Ry(-1.5002) ├
   └────────┘└─────────────┘


In [10]:
boltz_angle = 2 * np.arccos(np.sqrt(boltzmann_weight(omega))) #* Angle!

qr_boltz = QuantumRegister(1, 'boltz')
cr_boltz = ClassicalRegister(1, 'cr_boltz')
W_circ = QuantumCircuit(qr_boltz, cr_boltz)
W_circ.barrier()
W_circ.ry(boltz_angle, 0)
W_circ.x(0)
W_circ.barrier()


W_circ_op = Operator(W_circ)
# Distance to actual W
print(np.linalg.norm(W_circ_op.data - W_matrix))


0.0


In [11]:
boltz_angle = - 2 * np.arccos(np.sqrt(boltzmann_weight(omega))) #* Angle!

qr_boltz = QuantumRegister(1, 'boltz')
cr_boltz = ClassicalRegister(1, 'cr_boltz')
W_circ = QuantumCircuit(qr_boltz, cr_boltz)
W_circ.barrier()
W_circ.x(0)
W_circ.ry(boltz_angle, 0)
W_circ.barrier()


W_circ_op = Operator(W_circ)
# Distance to actual W
print(np.linalg.norm(W_circ_op.data - W_matrix))

0.0
